In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt

# 데이터 디렉토리 경로
data_dir = "/content/gdrive/My Drive/deeplearningproject/결과"

# 랜덤 시드 고정
random_seed = 42
tf.random.set_seed(random_seed)
np.random.seed(random_seed)

# 데이터 로드
batch_size = 32
img_height = 28
img_width = 28

dataset = image_dataset_from_directory(
    data_dir,
    image_size=(img_height, img_width),
    label_mode='categorical',
    batch_size=batch_size
)

# 클래스 비율 확인
class_names = dataset.class_names
print(f"Class names: {class_names}")

class_counts = np.zeros(len(class_names))
for _, labels in dataset:
    class_counts += np.sum(labels.numpy(), axis=0)

print(f"Class distribution: {class_counts}")

# 데이터셋을 numpy array로 변환
def dataset_to_numpy(dataset):
    images = []
    labels = []
    for img_batch, label_batch in dataset:
        images.append(img_batch.numpy())
        labels.append(label_batch.numpy())
    return np.concatenate(images), np.concatenate(labels)

images, labels = dataset_to_numpy(dataset)

# 데이터셋을 3:1:1 비율로 나누기
train_images, temp_images, train_labels, temp_labels = train_test_split(
    images, labels, test_size=0.4, stratify=labels, random_state=random_seed)

val_images, test_images, val_labels, test_labels = train_test_split(
    temp_images, temp_labels, test_size=0.5, stratify=temp_labels, random_state=random_seed)




In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 증강 생성기 정의
datagen = ImageDataGenerator(
    # rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    # shear_range=0.2,
    zoom_range=0.2,
    # horizontal_flip=True,
    fill_mode='nearest'
)

# 증강된 데이터를 이용한 학습 데이터셋 생성
def augment_dataset(images, labels, datagen, batch_size):
    augmented_images = []
    augmented_labels = []
    for img, label in zip(images, labels):
        img = img.reshape((1,) + img.shape)
        aug_iter = datagen.flow(img, batch_size=1)
        aug_img = aug_iter.next().astype(np.float32)
        augmented_images.append(aug_img.reshape(img.shape[1:]))
        augmented_labels.append(label)
    return np.array(augmented_images), np.array(augmented_labels)

augmented_train_images, augmented_train_labels = augment_dataset(train_images, train_labels, datagen, batch_size)

# numpy array를 tf.data.Dataset으로 변환

In [ ]:
# numpy array를 tf.data.Dataset으로 변환
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(batch_size)
augmented_train_dataset = tf.data.Dataset.from_tensor_slices((augmented_train_images, augmented_train_labels)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

In [ ]:
# 각 데이터셋의 클래스 비율 확인
def get_class_distribution(labels):
    class_counts = np.sum(labels, axis=0)
    return class_counts

print(f"Train class distribution: {get_class_distribution(train_labels)}")
print(f"Validation class distribution: {get_class_distribution(val_labels)}")
print(f"Test class distribution: {get_class_distribution(test_labels)}")

# 데이터셋에서 이미지 shape 확인
for images, labels in train_dataset.take(1):
    print(f"Train Image batch shape: {images.shape}")
    print(f"Train Label batch shape: {labels.shape}")
    plt.figure(figsize=(10, 10))
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[np.argmax(labels[i])])
        plt.axis("off")
    plt.show()

for images, labels in val_dataset.take(1):
    print(f"Validation Image batch shape: {images.shape}")
    print(f"Validation Label batch shape: {labels.shape}")

for images, labels in test_dataset.take(1):
    print(f"Test Image batch shape: {images.shape}")
    print(f"Test Label batch shape: {labels.shape}")

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

checkpoint_dir = "/content/gdrive/My Drive/deeplearningproject/best_model"

# 모델 정의
def create_baseline1_model():
    model = models.Sequential([
        layers.InputLayer(input_shape=(28, 28, 3)),
        layers.Conv2D(8, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.AveragePooling2D((2, 2)),

        layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.AveragePooling2D((2, 2)),

        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.AveragePooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(512, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# VGG 모델 정의
def create_vgg_model():
    model = models.Sequential([
        layers.InputLayer(input_shape=(28, 28, 3)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(512, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# DenseNet 모델 정의
def create_densenet_model():
    def dense_block(x, blocks, growth_rate):
        for _ in range(blocks):
            x = conv_block(x, growth_rate)
        return x

    def conv_block(x, growth_rate):
        x1 = layers.BatchNormalization()(x)
        x1 = layers.ReLU()(x1)
        x1 = layers.Conv2D(4 * growth_rate, (1, 1), use_bias=False)(x1)
        x1 = layers.BatchNormalization()(x1)
        x1 = layers.ReLU()(x1)
        x1 = layers.Conv2D(growth_rate, (3, 3), padding='same', use_bias=False)(x1)
        x = layers.Concatenate()([x, x1])
        return x

    def transition_block(x, reduction):
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        x = layers.Conv2D(int(x.shape[-1] * reduction), (1, 1), use_bias=False)(x)
        x = layers.AveragePooling2D((2, 2), strides=2)(x)
        return x

    inputs = layers.Input(shape=(28, 28, 3))
    x = layers.Conv2D(64, (3, 3), padding='same', use_bias=False)(inputs)
    x = dense_block(x, blocks=4, growth_rate=12)
    x = transition_block(x, reduction=0.5)
    x = dense_block(x, blocks=4, growth_rate=12)
    x = transition_block(x, reduction=0.5)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(2, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# ResNet 모델 정의
def create_resnet_model():
    def residual_block(x, filters, kernel_size=3, stride=1, conv_shortcut=True):
        shortcut = x
        if conv_shortcut:
            shortcut = layers.Conv2D(filters, (1, 1), strides=stride)(shortcut)
            shortcut = layers.BatchNormalization()(shortcut)

        x = layers.Conv2D(filters, kernel_size, padding='same', strides=stride)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)

        x = layers.Conv2D(filters, kernel_size, padding='same')(x)
        x = layers.BatchNormalization()(x)

        x = layers.add([shortcut, x])
        x = layers.ReLU()(x)
        return x

    inputs = layers.Input(shape=(28, 28, 3))
    x = layers.Conv2D(64, (3, 3), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = residual_block(x, 64)
    x = residual_block(x, 64)
    x = layers.MaxPooling2D((2, 2))(x)

    x = residual_block(x, 128, stride=2)
    x = residual_block(x, 128)
    x = layers.MaxPooling2D((2, 2))(x)

    x = residual_block(x, 256, stride=2)
    x = residual_block(x, 256)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(2, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


#inception 모델 정의
def create_inception_model(input_shape=(28, 28, 3), num_classes=2):
    base_model = InceptionV3(weights=None, include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model



def train_and_evaluate_model(model, train_dataset, val_dataset, test_dataset):
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=0.00001)
    checkpoint_path_ensemble = os.path.join(checkpoint_dir, "best_model_ensemble.h5")
    checkpoint_ensemble = tf.keras.callbacks.ModelCheckpoint(checkpoint_path_ensemble, monitor='val_loss', save_best_only=True, mode='min')

    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=100,
        batch_size=20,
        callbacks=[checkpoint_ensemble,reduce_lr],
        verbose=1
    )

    model.load_weights(checkpoint_path_ensemble)

    loss, accuracy = model.evaluate(test_dataset)
    print(f"Test loss: {loss}")
    print(f"Test accuracy: {accuracy}")

    y_true = np.concatenate([y for x, y in test_dataset], axis=0)
    y_pred = model.predict(test_dataset)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_true, axis=1)

    report = classification_report(y_true_classes, y_pred_classes, target_names=class_names, output_dict=True)
    return history, report, y_pred, y_true_classes

def calculate_ece(y_true, y_prob, n_bins=10):
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    bin_indices = np.digitize(np.max(y_prob, axis=1), bins) - 1
    ece = 0.0
    for i in range(n_bins):
        bin_mask = bin_indices == i
        bin_size = np.sum(bin_mask)
        if bin_size > 0:
            bin_accuracy = np.mean(y_true[bin_mask] == np.argmax(y_prob[bin_mask], axis=1))
            bin_confidence = np.mean(np.max(y_prob[bin_mask], axis=1))
            ece += np.abs(bin_accuracy - bin_confidence) * bin_size / len(y_true)
    return ece

def calculate_oe(y_true, y_prob, n_bins=10):
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    bin_indices = np.digitize(np.max(y_prob, axis=1), bins) - 1
    oe = 0.0
    for i in range(n_bins):
        bin_mask = bin_indices == i
        bin_size = np.sum(bin_mask)
        if bin_size > 0:
            bin_accuracy = np.mean(y_true[bin_mask] == np.argmax(y_prob[bin_mask], axis=1))
            bin_confidence = np.mean(np.max(y_prob[bin_mask], axis=1))
            if bin_confidence > bin_accuracy:
                oe += (bin_confidence - bin_accuracy) * bin_size / len(y_true)
    return oe

# 모델 1 (Baseline1)
model1 = create_baseline1_model()
history1, report1, y_pred1, y_true_classes = train_and_evaluate_model(model1, train_dataset, val_dataset, test_dataset)
# history1, report1, y_pred1, y_true_classes = train_and_evaluate_model(model1, augmented_train_dataset, val_dataset, test_dataset)

# 모델 2 (VGG)
model2 = create_vgg_model()
history2, report2, y_pred2, _ = train_and_evaluate_model(model2, train_dataset, val_dataset, test_dataset)
# history2, report2, y_pred2, _ = train_and_evaluate_model(model2, augmented_train_images, val_dataset, test_dataset)

# 모델 3 (DenseNet)
model3 = create_densenet_model()
history3, report3, y_pred3, _ = train_and_evaluate_model(model3, train_dataset, val_dataset, test_dataset)
# history3, report3, y_pred3, _ = train_and_evaluate_model(model3, train_dataset, val_dataset, test_dataset)

# 모델 4 (ResNet)
model4 = create_resnet_model()
history4, report4, y_pred4, _ = train_and_evaluate_model(model4, train_dataset, val_dataset, test_dataset)
# history4, report4, y_pred4, _ = train_and_evaluate_model(model4, augmented_train_images, val_dataset, test_dataset)

# 모델 5 (inception)
model5 = create_resnet_model()
history5, report5, y_pred5, _ = train_and_evaluate_model(model5, train_dataset, val_dataset, test_dataset)
# history5, report5, y_pred5, _ = train_and_evaluate_model(model5, augmented_train_images, val_dataset, test_dataset)


# 앙상블 예측
y_pred_ensemble = (y_pred1 + y_pred2 + y_pred3 + y_pred4 + y_pred5) / 5
y_pred_classes_ensemble = np.argmax(y_pred_ensemble, axis=1)

# 앙상블 분류 보고서 생성
report_ensemble = classification_report(y_true_classes, y_pred_classes_ensemble, target_names=class_names, output_dict=True)
accuracy_ensemble = report_ensemble['accuracy']
recall_ensemble = report_ensemble['weighted avg']['recall']
precision_ensemble = report_ensemble['weighted avg']['precision']
f1_score_ensemble = report_ensemble['weighted avg']['f1-score']

print(f"Ensemble Accuracy: {accuracy_ensemble}")
print(f"Ensemble Recall: {recall_ensemble}")
print(f"Ensemble Precision: {precision_ensemble}")
print(f"Ensemble F1-score: {f1_score_ensemble}")

# ECE와 OE 계산
ece_ensemble = calculate_ece(y_true_classes, y_pred_ensemble)
oe_ensemble = calculate_oe(y_true_classes, y_pred_ensemble)

print(f"Ensemble ECE: {ece_ensemble}")
print(f"Ensemble OE: {oe_ensemble}")

# 학습 과정 시각화
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history1.history['loss'], label='Baseline1 Train Loss')
plt.plot(history1.history['val_loss'], label='Baseline1 Valid Loss')
plt.plot(history2.history['loss'], label='VGG-like Train Loss')
plt.plot(history2.history['val_loss'], label='VGG-like Valid Loss')
plt.plot(history3.history['loss'], label='DenseNet-like Train Loss')
plt.plot(history3.history['val_loss'], label='DenseNet-like Valid Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history1.history['accuracy'], label='Baseline1 Train Accuracy')
plt.plot(history1.history['val_accuracy'], label='Baseline1 Valid Accuracy')
plt.plot(history2.history['accuracy'], label='VGG-like Train Accuracy')
plt.plot(history2.history['val_accuracy'], label='VGG-like Valid Accuracy')
plt.plot(history3.history['accuracy'], label='DenseNet-like Train Accuracy')
plt.plot(history3.history['val_accuracy'], label='DenseNet-like Valid Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()
